In [1]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity

class RecommendationModel:
    def __init__(self, user_data, ratings_data, product_data):
        # Store the datasets in attributes
        self.user_data = user_data
        self.ratings_data = ratings_data
        self.product_data = product_data

    @staticmethod
    def calculate_similarity(user1, user2):
        """Calculate similarity between two users."""
        gender_sim = 1 if user1['gender'] == user2['gender'] else 0
        age_sim = 1 - abs(user1['age'] - user2['age']) / 100
        if user1['city'] == user2['city']:
            location_sim = 1
        elif user1['country'] == user2['country']:
            location_sim = 0.5
        else:
            location_sim = 0
        total_similarity = 0.2 * gender_sim + 0.6 * age_sim + 0.2 * location_sim
        return total_similarity

    def find_top5_similar_users(self, user_info):
        """Find the top 5 similar users based on user attributes."""
        similarities = []
        for i, other_user in self.user_data.iterrows():
            similarity = self.calculate_similarity(user_info, other_user)
            similarities.append((other_user['userID'], similarity))
        similarities.sort(key=lambda x: x[1], reverse=True)
        return [user[0] for user in similarities[:5]]

    def collaborative_filtering_recommendations(self, user_info, top_n=10):
        """Generate recommendations using collaborative filtering."""
        top5_similar_users = self.find_top5_similar_users(user_info)
        filtered_ratings_data = self.ratings_data[self.ratings_data['userid'].isin(top5_similar_users)]
        user_item_matrix = filtered_ratings_data.pivot_table(
            index='userid',
            columns='productid',
            values='rating',
            aggfunc='mean'
        ).fillna(0)
        user_similarity = cosine_similarity(user_item_matrix)
        recommended_items = set()
        for target_user_id in top5_similar_users:
            try:
                target_user_index = user_item_matrix.index.get_loc(target_user_id)
            except KeyError:
                continue
            user_similarities = user_similarity[target_user_index]
            similar_users_indices = user_similarities.argsort()[::-1][1:]
            for user_index in similar_users_indices:
                rated_by_similar_user = user_item_matrix.iloc[user_index]
                not_rated_by_target_user = (rated_by_similar_user > 0) & (user_item_matrix.iloc[target_user_index] == 0)
                recommended_items.update(user_item_matrix.columns[not_rated_by_target_user][:top_n])
                if len(recommended_items) >= top_n:
                    break
        recommended_items_details = self.ratings_data[
            self.ratings_data['productid'].isin(recommended_items)
        ][['productid', 'rating']].drop_duplicates()
        recommended_items_with_names = recommended_items_details.merge(
        self.product_data[['id', 'name']],
        left_on='productid',
        right_on='id',
        how='inner')
        top_recommendations = recommended_items_with_names.sort_values(by='rating', ascending=False ).head(top_n)
        return top_recommendations[['productid', 'name']].values.tolist()

    def highest_rated_products_by_interest(self, user_info, top_n=20):
        """Get top-rated products based on user interests."""
        user_interest = list(map(int, user_info['interest'].split(',')))
        filtered_products = self.product_data[self.product_data['main_category_encoded'].isin(user_interest)]
        top_products = filtered_products.sort_values(
            by=['ratings', 'no_of_ratings'], ascending=False
        ).head(top_n)
        return top_products[['id', 'name']].values.tolist()

    def recommend(self, user_info, top_n=20):
        """Generate final recommendations combining collaborative filtering and interest-based filtering."""
        collaborative_recommendations = self.collaborative_filtering_recommendations(user_info, top_n=top_n)
        interest_based_recommendations = self.highest_rated_products_by_interest(user_info, top_n=top_n)

        recommendations = {
            "collaborative": collaborative_recommendations,
            "interest_based": interest_based_recommendations
        }
        return recommendations

    def save(self, filepath):
        """Save the model to a file."""
        with open(filepath, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load(filepath):
        """Load the model from a file."""
        with open(filepath, 'rb') as f:
            return pickle.load(f)


KeyboardInterrupt: 

In [2]:
# Usage Example
if __name__ == "__main__":
    user_data = pd.read_csv('../user_data.csv')
    ratings_data = pd.read_csv('../ratings_test.csv')
    product_data = pd.read_csv('../test_data.csv')
    # Initialize the model
    model = RecommendationModel(user_data, ratings_data, product_data)
    # Save the model to a file
    model.save('D:/RecommendModel/Saved_model/interest_recommendation_model.pkl')
    # Load the model from the file
    # Example user info
    user_info = {
        'age': 22,
        'gender': 'female',
        'city': 'thomasville',
        'country': 'us',
        'interest': '18,19'
    }
    # Generate recommendations
    recommendation_interest = model.recommend(user_info, top_n=20)
    print("Collaborative Recommendations:", recommendation_interest['collaborative'])
    print("Interest-Based Recommendations:", recommendation_interest['interest_based'])

Collaborative Recommendations: [['app250928', 'Black + Decker 2000 Watts Ceramic Room Heater with Digital Control- Black'], ['acs106891', "HUSIRE'S Scrunchies Watch Classic & Unique Scrunchies Watches Strap Cloth Soft Pattern Solid Fabric Wristband I Watch Cute..."], ['acs245060', 'The A P Exim Commercials Reusable Jacquard Cotton Boxy Tote Bag for Women | Printed Multipurpose Handbag | Box Bag |Tote B...'], ['ggf196312', "Shara's Dry Fruits Broken Cashews Nuts Kaju Tukda, 250g - 4 Pieces"], ['app253530', 'Wonderchef Electric Kettle Crimson Edge, Stainless Steel, With Auto-Shut Off, 2-Level Safety, Cool-Touch, 1.7 Litres, 1 Ye...'], ['acs157229', 'Delsey Banjul Denier Polyester 78 CM 4 Wheels Red Soft Suitcase'], ['acs010207', 'Zivom® Leather Silver Plated Bracelet for Unisex Adult'], ['hkt067539', 'Oc9 Set of 4 Measuring Cup and 4 Measuring Spoon Silver (Stainless Steel)'], ['hkt204108', 'AMFIN® (Pack of 8) Happy Birthday Printed Pastel Color Candle with Holder for Birthday Cake Toppe

In [4]:
# import pickle
# user_data = pd.read_csv('user_data.csv')
# ratings_data = pd.read_csv('ratings_test.csv')
# product_data = pd.read_csv('test_data.csv')
# Assuming the class is named RecommendationModel and initialized like this:
model = RecommendationModel(
    user_data=user_data,
    product_data=product_data,
    ratings_data=ratings_data
)

# Save the model to a pickle file
with open('D:/RecommendModel/Saved_model/recommendation_interest_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model saved to recommendation_interest_model.pkl")


Model saved to recommendation_interest_model.pkl
